### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- open a list of raster files
- convert and save all rasters as ASCII

### Prerequisites
- Rasters should be in the appropriate CRS, as no projection is used
- rasterio

# init

In [ ]:
from pathlib import Path
import rasterio
import matplotlib.pyplot as plt

from rasterio import features
import numpy as np
from rasterio.plot import show

def get_coords(gdf):
    coords = gdf.envelope
    epsg_coords = coords #.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    #roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    #rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    #coi = roi.centroid(maxError=1)
    return epsg_coords, transform #roi, rec_roi, coi

# imports

In [ ]:
# import a list of raster files
# all files should be in the same folder and share the projection

raster_folder = Path('../data/processed/')
MUM_raster_list = ['']

JAK_raster_list = ['']

MAN_raster_list = ['']

In [ ]:
# define a function that opens a list of raster files and stores them in a list (should it be a dict?)
# the function should also print the crs of the raster files
def open_raster_list(raster_list, raster_folder):
    open_rasters = {}
    array_list = []
    crs_list = []
    transform_list = []
    for raster in raster_list: #[0:3]
        raster_path = raster_folder / raster
        raster_name = raster.split('.')[0]
        array_list.append(raster_name)

    for i in range(len(array_list)):
        with rasterio.open(raster_path, 'r') as array_list[i]:
            array = array_list[i].read(1)
            crs = array_list[i].crs
            transform = array_list[i].transform
            print(array_list[i].crs)
            #array_list.append(array)
            #crs_list.append(crs)
            #transform_list.append(transform)
            # update the dict open_rasters with the arrays, crs and transform
            open_rasters[array_list[i]['array']] = array
            open_rasters[array_list[i]['crs']] = crs
            open_rasters[array_list[i]['transform']] = transform
    return open_rasters

In [ ]:
# import the rasters for each city at a time
input_which_city = input('Which city do you want to import the rasters for? (1 = MUM, 2 = JAK, 3 = MAN:')
if input_which_city == '1':
    raster_list = MUM_raster_list
elif input_which_city == '2':
    raster_list = JAK_raster_list
elif input_which_city == '3':
    raster_list = MAN_raster_list
else:
    print('Please enter a valid city number')
    exit()

list_open_raster = open_raster_list(raster_list, raster_folder)


In [ ]:
list_open_raster

# analysis

In [ ]:
# convert each raster in list_open_raster to a numpy array




# export

In [ ]:
# define two export functions
# export rasters as geotiff
def export_raster(raster,url):
    with rasterio.open(
        url, "w",
        driver = "GTiff",
        crs = ref_raster.crs,
        transform = ref_raster.transform,
        dtype = rasterio.float32,
        count = 1,
        width = ref_raster.width,
        height = ref_raster.height) as dst:
        dst.write(raster, indexes = 1)

# export raster as ascii files
def export_raster_to_ascii(raster_obj, export_path, raster_data_type):
    with rasterio.open(export_path, "w", driver='AAIGrid',
                    height = ref_raster.shape[0], width=ref_raster.shape[1],
                    count=1, dtype=list(ref_raster.dtypes)[0],
                    crs=ref_raster.crs, transform = ref_raster.transform) as dest:
        dest.write(raster_obj.astype(raster_data_type), indexes=1)

In [ ]:
# export the calculated arrays as raster files in geotiff format
out_path = Path('../data/processed/')
distance_url = out_path/distance_raster_file
major_roads_url = out_path/major_road_raster_file
full_roads_url = out_path/full_road_raster_file

export_raster(distance_to_roads,distance_url)
export_raster(major_roads_raster,major_roads_url)
export_raster(full_roads_raster,full_roads_url)

In [ ]:
# export the calculated arrays as ASCII files
# define the ASCII file names
out_path = Path('../data/processed/')

distance_ascii_url = out_path/(distance_raster_file.split('.')[0] + '.asc')
major_roads_ascii_url = out_path/(major_road_raster_file.split('.')[0] + '.asc')
full_roads_ascii_url = out_path/(full_road_raster_file.split('.')[0] + '.asc')

# call the ASCII export function
export_raster_to_ascii(distance_to_roads, distance_ascii_url,rasterio.float32)
export_raster_to_ascii(major_roads_raster, major_roads_ascii_url,rasterio.int16)
export_raster_to_ascii(full_roads_raster, full_roads_ascii_url,rasterio.int16)
